In [24]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

import plot_boundary
from kernel_SVM_functions import rbf_kernel
from classifiers import SoftMarginKernelClassifier, QUBOSoftMarginClassifier
from parameter_tuning import tune_csvm_parameters

In [23]:
"""Want to conduct analysis on synth_0.2, synth_0.3, synth_0.4"""
filename = 'synth_0.4.csv'

data = np.loadtxt(f'synth_data/{filename}', delimiter = ',')

In [25]:
X = data[:, :-1]
t = data[:, -1]

X_train, X_test, t_train, t_test = train_test_split(X, t, train_size = 0.15, shuffle = True, stratify = t)

In [27]:
X_train.shape

(18, 2)

In [28]:
72**2

5184

In [26]:
C_values = [2 ** i for i in range(0, 7)]
gamma_values = [-1] + [2 **i for i in range(-3, 3)]
kernel_func = rbf_kernel

acc, auroc, times = tune_csvm_parameters(X_train, t_train, C_values, gamma_values, kernel_func, k_folds = 10)

Progress:   0%|          | 0/49 [00:00<?, ?it/s]


ValueError: n_splits=10 cannot be greater than the number of members in each class.